In [1]:
from openai import OpenAI
import os
from google import genai
from google.genai import types as genTypes
from vif.falcon.oracle.guided_oracle.guided_code_oracle import OracleGuidedCodeModule
from vif.utils.renderer.tex_renderer import TexRenderer

gclient = genai.Client(
    api_key=os.environ.get("GOOGLE_API_KEY"),
    http_options=genTypes.HttpOptions(api_version="v1alpha"),
)

oclient = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.environ.get("OPENROUTER_API_KEY"),
)


oracle_module = OracleGuidedCodeModule(
    model="meta-llama/llama-4-maverick:free",
    temperature=0.3,
    client=oclient,
    visual_client=gclient,
    visual_model="gemini-2.5-flash",
    property_client=oclient,
    property_model="mistralai/mistral-small-3.2-24b-instruct:free",
    property_model_temperature=0.3
)

renderer = TexRenderer()


/home/creux/Documents/AI/VIFagent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-07 11:35:33.747 | INFO     | vif.utils.caching:instantiate_cache:26 - seg_cache cache loaded


In [3]:
from datasets import load_dataset


ds = load_dataset("CharlyR/vtikz", "tikz", split="benchmark")
ds = ds.select_columns(["id","type","instruction","code","image_solution","image_input","code_solution"])

Generating test split: 100%|██████████| 2/2 [00:00<00:00, 766.43 examples/s]


In [8]:
import_str = """from vif.falcon.oracle.guided_oracle.property_expression import visual_property
from vif.falcon.oracle.guided_oracle.expressions import (
    OracleExpression,
    aligned,
    present,
    angle,
    placement,
    position,
    color,
    shape,
    size,
    within,
    mirrored,
)
"""
for row in ds.iter(1):
    if os.path.exists(
        f"notebooks/oracle_creation/llm_oracle_functions/{row["id"][0]}.py"
    ):
        continue
    with open(
        f"notebooks/oracle_creation/llm_oracle_functions/{row["id"][0]}.py", "w"
    ) as oracle_file:
        image = renderer.from_string_to_image(row["code"][0])
        oracle_code,_ = oracle_module.get_oracle_code(row["instruction"][0],base_image=image)
        
        oracle_file.write(import_str)
        oracle_file.write(oracle_code)
    

2025-10-07 11:38:18.888 | DEBUG    | vif.utils.renderer.tex_renderer:from_string_to_image:45 - latex renderer writing to /home/creux/.cache/varbench/0106df1e-0bd3-4f52-8f87-94a32953ce47.tex
2025-10-07 11:38:19.670 | DEBUG    | vif.utils.renderer.tex_renderer:from_string_to_image:78 - converting /home/creux/.cache/varbench/0106df1e-0bd3-4f52-8f87-94a32953ce47.tex to png
2025-10-07 11:38:25.574 | DEBUG    | vif.utils.renderer.tex_renderer:from_string_to_image:45 - latex renderer writing to /home/creux/.cache/varbench/8509e93c-d305-46cd-9df0-bdae6df1ae56.tex
2025-10-07 11:38:25.974 | DEBUG    | vif.utils.renderer.tex_renderer:from_string_to_image:78 - converting /home/creux/.cache/varbench/8509e93c-d305-46cd-9df0-bdae6df1ae56.tex to png
2025-10-07 11:38:31.286 | DEBUG    | vif.utils.renderer.tex_renderer:from_string_to_image:45 - latex renderer writing to /home/creux/.cache/varbench/07ea02e0-6095-49e4-bd5c-45622bf47c66.tex
2025-10-07 11:38:31.694 | DEBUG    | vif.utils.renderer.tex_render